In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - 1.1_sample_anisotropic_dfs.ipynb
# AUTHOR - James Lane
# PROJECT - tng-dfs
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Draw samples and plot fundamental properties of the anisotropic DFs
'''

__author__ = "James Lane"

In [ ]:
# %load ../../src/nb_modules/nb_imports.txt
### Imports

## Basic
import numpy as np
import sys, os
import dill as pickle
import time, logging

## Plotting
import matplotlib as mpl
from matplotlib import pyplot as plt

## Astropy
from astropy import units as apu

## galpy
from galpy import potential

## Project-specific
src_path = 'src/'
while True:
    if os.path.exists(src_path): break
    if os.path.realpath(src_path).split('/')[-1] in ['tng-dfs','/']:
            raise FileNotFoundError('Failed to find src/ directory.')
    src_path = os.path.join('..',src_path)
sys.path.insert(0,src_path)
from tng_dfs import cutout as pcutout
from tng_dfs import densprofile as pdens
from tng_dfs import fitting as pfit
from tng_dfs import io as pio
from tng_dfs import kinematics as pkin
from tng_dfs import util as putil

### Notebook setup

%matplotlib inline
plt.style.use(os.path.join(src_path,'mpl/project.mplstyle')) # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

### Keywords, loading, pathing

In [ ]:
# %load ../../src/nb_modules/nb_setup.txt
# Keywords
cdict = putil.load_config_to_dict()
keywords = ['DATA_DIR','MW_ANALOG_DIR','FIG_DIR_BASE','FITTING_DIR_BASE',
            'RO','VO','ZO','LITTLE_H','MW_MASS_RANGE']
data_dir,mw_analog_dir,fig_dir_base,fitting_dir_base,ro,vo,zo,h,\
    mw_mass_range = putil.parse_config_dict(cdict,keywords)

# MW Analog 
mwsubs,mwsubs_vars = putil.prepare_mwsubs(mw_analog_dir,h=h,
    mw_mass_range=mw_mass_range,return_vars=True,force_mwsubs=False,
    bulge_disk_fraction_cuts=True)

# Figure path
local_fig_dir = './fig/'
fig_dir = os.path.join(fig_dir_base, 
    'notebooks/5_compare_distribution_functions/1.1_sample_anisotropic_dfs')
os.makedirs(local_fig_dir,exist_ok=True)
os.makedirs(fig_dir,exist_ok=True)
show_plots = False

# Load tree data
tree_primary_filename = os.path.join(mw_analog_dir,
    'major_mergers/tree_primaries.pkl')
with open(tree_primary_filename,'rb') as handle: 
    tree_primaries = pickle.load(handle)
tree_major_mergers_filename = os.path.join(mw_analog_dir,
    'major_mergers/tree_major_mergers.pkl')
with open(tree_major_mergers_filename,'rb') as handle:
    tree_major_mergers = pickle.load(handle)
n_mw = len(tree_primaries)

### Plotting functions

In [ ]:
def plot_ELz(nbody_orbs, sample_orbs, nbody_E, pot, interpot=None, 
    plot_hist=True, fig=None, axs=None, cmap=mpl.colormaps.get_cmap('rainbow')):
    '''Plot energy and angular momentum'''

    # Some plotting kwargs
    Lz_range = [-3,3]
    label_fs = 12
    E_range = np.percentile(nbody_E, [0.5,99.5])

    # Set up figure
    if fig is None or axs is None:
        fig = plt.figure(figsize=(12,3))
        axs = fig.subplots(nrows=1, ncols=3)

    # N-body properties
    nbody_Lz = nbody_orbs.Lz().to_value(apu.kpc*apu.km/apu.s)

    # Sample properties
    sample_Lz = sample_orbs.Lz().to_value(apu.kpc*apu.km/apu.s)
    sample_potE = sample_orbs.E(pot=pot).to_value(apu.km**2/apu.s**2)
    sample_interE = sample_orbs.E(pot=interpot).to_value(apu.km**2/apu.s**2)

    # Angular momentum vs energy
    Lzs = [nbody_Lz, sample_Lz, sample_Lz]
    energies = [nbody_E, sample_potE, sample_interE]
    labels = ['N-body',
            'Samples (interpolated pot)', 
            'Samples (Best-fit pot)']
    for k in range(3):
        E_range = [np.nanmin(energies[k])/1e5, np.nanmax(energies[k])/1e5]
        if plot_hist:
            H, xedges, yedges = np.histogram2d(Lzs[k]/1e3, energies[k]/1e5, 
                bins=[45,30], range=[Lz_range,E_range])
            H = np.rot90(H)
            H = np.flipud(H)
            Hmasked = np.log10(np.ma.masked_where(H==0,H))
            if k == 0:
                vmin = np.nanmin(Hmasked)
                vmax = np.nanmax(Hmasked)
            cmap.set_bad(color='white')
            axs[k].pcolormesh(xedges,yedges,Hmasked,cmap=cmap,vmin=vmin,
                vmax=vmax)        
        else:
            axs[k].scatter(Lzs[k], energies[k], s=1, color='Black', alpha=0.1)
        
        # Decorate
        axs[k].axvline(0, linestyle='dashed', linewidth=1., color='Grey')
        _annotate_bbox_kwargs = dict(facecolor='White', edgecolor='Black', 
            fill=True, alpha=0.5)
        axs[k].annotate(labels[k], xy=(0.05,0.05), xycoords='axes fraction',
            fontsize=8, bbox=_annotate_bbox_kwargs)
        axs[k].set_xlabel(r'Lz [$10^{3}$ kpc km/s]', fontsize=label_fs)
        axs[k].set_ylabel(r'E [$10^{5}$ km$^{2}$/s$^{2}$]', fontsize=label_fs)
        
        axs[k].set_xlim(Lz_range[0],Lz_range[1])
        axs[k].set_ylim(E_range[0],E_range[1])

    fig.tight_layout()
    return fig, axs

def plot_beta_vdisp(nbody_orbs, sample_orbs, n_bs=100, fig=None, axs=None):
    '''Plot the beta and then the velocity dispersion'''

    # Some kwargs for plotting
    nbody_color = 'Black'
    sample_color = 'DodgerBlue'

    # Set up figure
    if fig is None or axs is None:
        fig = plt.figure(figsize=(5,12))
        axs = fig.subplots(nrows=4, ncols=1)

    # Binning for velocity dispersions and betas
    n_bin = np.min([500, len(nbody_orbs)//10]) # n per bin
    adaptive_binning_kwargs = {
        'n':n_bin,
        'rmin':0.,
        'rmax':np.max( nbody_orbs.r().to_value(apu.kpc) ),
        'bin_mode':'exact numbers',
        'bin_equal_n':True,
        'end_mode':'ignore',
        'bin_cents_mode':'median',
    }
    bin_edges, bin_cents, bin_n = pkin.get_radius_binning(nbody_orbs, 
        **adaptive_binning_kwargs)

    # Compute velocity dispersions for N-body
    compute_betas_kwargs = {'use_dispersions':True,
                            'return_kinematics':True}
    nbody_beta, nbody_vr2, nbody_vp2, nbody_vz2 = \
        pkin.compute_betas_bootstrap(nbody_orbs, bin_edges, n_bootstrap=n_bs, 
        compute_betas_kwargs=compute_betas_kwargs)

    # Compute velocity dispersions for the DF samples
    compute_betas_kwargs = {'use_dispersions':True,
                            'return_kinematics':True}
    sample_beta, sample_vr2, sample_vp2, sample_vz2 = \
        pkin.compute_betas_bootstrap(sample_orbs, bin_edges, n_bootstrap=n_bs, 
        compute_betas_kwargs=compute_betas_kwargs)

    # Beta for the N-body
    axs[0].plot(bin_cents, np.median(nbody_beta, axis=0), color=nbody_color, 
        label='N-body')
    axs[0].fill_between(bin_cents, np.percentile(nbody_beta, 16, axis=0),
        np.percentile(nbody_beta, 84, axis=0), color=nbody_color, alpha=0.25)

    # Beta for the DF samples
    axs[0].plot(bin_cents, np.median(sample_beta, axis=0), color=sample_color, 
        label='DF Samples')
    axs[0].fill_between(bin_cents, np.percentile(sample_beta, 16, axis=0),
        np.percentile(sample_beta, 84, axis=0), color=sample_color, alpha=0.25)

    # Velocity dispersions for the N-body
    sv2s = [nbody_vr2,nbody_vp2,nbody_vz2]
    v_suffixes = ['r','\phi','z']
    for k in range(3):
        axs[k+1].plot(bin_cents, np.sqrt(np.median(sv2s[k], axis=0)), 
            color=nbody_color)
        axs[k+1].fill_between(bin_cents, 
            np.sqrt(np.percentile(sv2s[k], 16, axis=0)),
            np.sqrt(np.percentile(sv2s[k], 84, axis=0)), 
            color=nbody_color, alpha=0.25)
        axs[k+1].set_ylabel(r'$\sigma_'+v_suffixes[k]+r'$')

    # Velocity dispersions for the DF samples
    sv2s = [sample_vr2,sample_vp2,sample_vz2]
    v_suffixes = ['r','\phi','z']
    for k in range(3):
        axs[k+1].plot(bin_cents, np.sqrt(np.median(sv2s[k], axis=0)), 
            color=sample_color)
        axs[k+1].fill_between(bin_cents, 
            np.sqrt(np.percentile(sv2s[k], 16, axis=0)),
            np.sqrt(np.percentile(sv2s[k], 84, axis=0)), 
            color=sample_color, alpha=0.25)
        axs[k+1].set_ylabel(r'$\sigma_'+v_suffixes[k]+r'$')

    # Labels
    axs[0].set_ylabel(r'$\beta$')
    axs[0].legend()
    for k in range(4):
        axs[k].set_xscale('log')
        axs[k].set_xlabel(r'$r$ [kpc]')
        if k > 0:
            axs[k].set_yscale('log')
    fig.tight_layout()
    
    return fig, axs

### Sample and plot all DFs
- Constant anisotropy
- Osipkov-Merritt
- Osipkov-Merritt linear combination

In [ ]:
verbose = True
dens_fitting_dir = os.path.join(fitting_dir_base,'density_profile')
df_fitting_dir = os.path.join(fitting_dir_base,'distribution_function')
analysis_version = 'v1.1'
analysis_dir = os.path.join(mw_analog_dir,'analysis',analysis_version)

# Begin logging
log_filename = './log/1.1_sample_anisotropic_dfs.log'
if os.path.exists(log_filename):
    os.remove(log_filename)
logging.basicConfig(filename=log_filename, level=logging.INFO, filemode='w', 
    force=True)
logging.info('Beginning anisotropic DF sample creation and plotting. Time: '+\
             time.strftime('%a, %d %b %Y %H:%M:%S',time.localtime()))

# Potential interpolator version
interpot_version = 'all_star_dm_enclosed_mass'

# DM halo information
dm_halo_version = 'poisson_nfw'
dm_halo_ncut = 500

# Stellar bulge and disk information
stellar_bulge_disk_version = 'miyamoto_disk_pswc_bulge_tps_halo'
stellar_bulge_disk_ncut = 2000

# Stellar halo density information
stellar_halo_density_version = 'poisson_twopower_softening'
stellar_halo_density_ncut = 500

# Stellar halo rotation information
stellar_halo_rotation_dftype = 'tanh_rotation'
stellar_halo_rotation_version = 'asymmetry_fit'
stellar_halo_rotation_ncut = 500

# Define density profiles
dm_halo_densfunc = pdens.NFWSpherical()
disk_densfunc = pdens.MiyamotoNagaiDisk()
bulge_densfunc = pdens.SinglePowerCutoffSpherical()
stellar_halo_densfunc = pdens.TwoPowerSpherical()
stellar_bulge_disk_densfunc = pdens.CompositeDensityProfile(
    [disk_densfunc,
     bulge_densfunc,
     pdens.TwoPowerSpherical()]
     )

sample_data_cb = []
sample_data_om = []
sample_data_om2 = []

for i in range(n_mw):
    # if i > 0: continue

    # Get the primary
    primary = tree_primaries[i]
    z0_sid = primary.subfind_id[0]
    major_mergers = primary.tree_major_mergers
    n_major = primary.n_major_mergers
    n_snap = len(primary.snapnum)
    primary_filename = primary.get_cutout_filename(mw_analog_dir,
        snapnum=primary.snapnum[0])
    co = pcutout.TNGCutout(primary_filename)
    co.center_and_rectify()
    pid = co.get_property('stars','ParticleIDs')

    # Get the dark halo
    dm_halo_filename = os.path.join(dens_fitting_dir,'dm_halo/',dm_halo_version,
        str(z0_sid), 'sampler.pkl')
    dm_halo_pot = pfit.construct_pot_from_fit(dm_halo_filename,
        dm_halo_densfunc, dm_halo_ncut, ro=ro, vo=vo)
    
    # Get the stellar bulge and disk
    stellar_bulge_disk_filename = os.path.join(dens_fitting_dir,
        'stellar_bulge_disk/',stellar_bulge_disk_version,str(z0_sid),
        'sampler.pkl')
    stellar_pots = pfit.construct_pot_from_fit(stellar_bulge_disk_filename,
        stellar_bulge_disk_densfunc, stellar_bulge_disk_ncut, ro=ro, vo=vo)
    fpot = [stellar_pots[1], stellar_pots[0], dm_halo_pot] # bulge, disk, halo

    # Load the interpolator for the sphericalized potential
    interpolator_filename = os.path.join(dens_fitting_dir,
        'spherical_interpolated_potential/',interpot_version,
        str(z0_sid),'interp_potential.pkl')
    with open(interpolator_filename,'rb') as handle:
        interpot = pickle.load(handle)

    for j in range(n_major):
        if verbose: 
            msg = f'Constructing DF samples for MW analog {i+1}/{n_mw}'+\
                  f' merger {j+1}/{n_major}'
            logging.info(msg)
            print(msg)

        # Get the major merger
        major_merger = primary.tree_major_mergers[j]
        major_acc_sid = major_merger.subfind_id[0]
        major_mlpid = major_merger.secondary_mlpid
        upid = major_merger.get_unique_particle_ids('stars',data_dir=data_dir)
        indx = np.where(np.isin(pid,upid))[0]
        orbs = co.get_orbs('stars')[indx]
        rs = orbs.r().to(apu.kpc).value
        masses = co.get_masses('stars')[indx].to_value(apu.Msun)
        n_star = len(orbs)
        star_mass = co.get_masses('stars')[indx].to_value(apu.Msun)
        pe = co.get_potential_energy('stars')[indx].to_value(apu.km**2/apu.s**2)
        vels = co.get_velocities('stars')[indx].to_value(apu.km/apu.s)
        vmag = np.linalg.norm(vels,axis=1)
        energy = pe + 0.5*vmag**2
        Lz = orbs.Lz().to_value(apu.kpc*apu.km/apu.s)

        # Mask the input orbits to only include those with radius greater
        # than the softening length
        r_softening = putil.get_softening_length('stars', z=0, physical=True)
        mask = rs > r_softening
        orbs = orbs[mask]
        rs = rs[mask]
        masses = masses[mask]
        n_star = len(orbs)
        star_mass = star_mass[mask]
        pe = pe[mask]
        vels = vels[mask]
        vmag = vmag[mask]
        energy = energy[mask]
        Lz = Lz[mask]

        # Get the stellar halo density profile (denspot for the DF)
        stellar_halo_density_filename = os.path.join(dens_fitting_dir,
            'stellar_halo/',stellar_halo_density_version,str(z0_sid),
            'merger_'+str(j+1)+'/', 'sampler.pkl')
        denspot = pfit.construct_pot_from_fit(
            stellar_halo_density_filename, stellar_halo_densfunc, 
            stellar_halo_density_ncut, ro=ro, vo=vo)
        
        # Get the stellar halo rotation kernel
        stellar_halo_rotation_filename = os.path.join(df_fitting_dir,
            stellar_halo_rotation_dftype,stellar_halo_rotation_version,
            str(z0_sid),'merger_'+str(j+1)+'/', 'sampler.pkl')
        stellar_halo_k, stellar_halo_chi = \
            pio.median_params_from_emcee_sampler(stellar_halo_rotation_filename,
                ncut=stellar_halo_rotation_ncut)
        
        ### Constant beta DF ###
        if verbose:
            msg = 'Sampling constant beta DF'
            logging.info(msg)
            print(msg)
        df_type = 'constant_beta'
        df_version = 'df_density_softening'
        try:
            # Load the distribution function and wrangle
            df_filename = os.path.join(df_fitting_dir,df_type,df_version,
                str(z0_sid),'merger_'+str(j+1),'df.pkl')
            with open(df_filename,'rb') as handle:
                dfcb = pickle.load(handle)
            dfcb = pkin.reconstruct_anisotropic_df(dfcb, interpot, denspot)
        
            # Create sample and apply rotation
            sample = dfcb.sample(n=n_star, rmin=rs.min()*apu.kpc*0.9)
            sample = pkin.rotate_df_samples(sample,stellar_halo_k,
                stellar_halo_chi)
        except Exception as e:
            if verbose:
                msg = 'Caught an error when loading DF / sampling: '+str(e)+\
                    ' continuing'
                logging.info(msg)
                print(msg)
            continue

        _data_cb = (z0_sid,
                    major_acc_sid,
                    major_mlpid,
                    j+1, # merger number
                    sample)
        sample_data_cb.append(_data_cb)

        # Scale the N-body energies by the potential energy of interpot at the 
        # stellar half-mass radius
        rhalf = pkin.half_mass_radius(rs, masses)
        rhalf_perc = 0.05
        rhalf_mask = np.abs(rs-rhalf) < rhalf_perc*rhalf
        rhalf_pe_star = np.median(pe[rhalf_mask])
        rhalf_pe_interpot = potential.evaluatePotentials(
            interpot, rhalf*apu.kpc, 0.).to_value(apu.km**2/apu.s**2)
        rhalf_pe_offset = rhalf_pe_star - rhalf_pe_interpot

        # Plotting
        this_fig_dir = os.path.join(fig_dir, df_type, df_version, str(z0_sid), 
            'merger_'+str(j+1))
        os.makedirs(this_fig_dir,exist_ok=True)

        fig,axs = plot_ELz(orbs, sample, energy-rhalf_pe_offset, fpot, 
            interpot)
        fig.tight_layout()
        figname = os.path.join(this_fig_dir,'energy_Lz.png')
        fig.savefig(figname, dpi=300)
        plt.close(fig)

        fig,axs = plot_beta_vdisp(orbs, sample)
        for ax in axs: ax.axvline(r_softening, linestyle='dashed', 
            linewidth=1., color='Grey')
        fig.tight_layout()
        figname = os.path.join(this_fig_dir,'velocity_dispersions.png')
        fig.savefig(figname, dpi=300)
        plt.close(fig)

        ### Osipkov-Merritt DF ###
        if verbose:
            msg = 'Sampling Osipkov-Merritt DF'
            logging.info(msg)
            print(msg)
        df_type = 'osipkov_merritt'
        df_version = 'df_density_softening'
        try:
            # Load the distribution function and wrangle
            df_filename = os.path.join(df_fitting_dir,df_type,df_version,
                str(z0_sid),'merger_'+str(j+1),'df.pkl')
            with open(df_filename,'rb') as handle:
                dfom = pickle.load(handle)
            dfom = pkin.reconstruct_anisotropic_df(dfom, interpot, denspot)
        
            # Create sample and apply rotation
            sample = dfom.sample(n=n_star, rmin=rs.min()*apu.kpc*0.9)
            sample = pkin.rotate_df_samples(sample,stellar_halo_k,
                stellar_halo_chi)
        except Exception as e:
            if verbose:
                msg = 'Caught an error when loading DF / sampling: '+str(e)+\
                    ' continuing'
                logging.info(msg)
                print(msg)
            continue

        _data_om = (z0_sid,
                    major_acc_sid,
                    major_mlpid,
                    j+1, # merger number
                    sample)
        sample_data_om.append(_data_om)

        # Scale the N-body energies by the potential energy of interpot at the 
        # stellar half-mass radius
        rhalf = pkin.half_mass_radius(rs, masses)
        rhalf_perc = 0.05
        rhalf_mask = np.abs(rs-rhalf) < rhalf_perc*rhalf
        rhalf_pe_star = np.median(pe[rhalf_mask])
        rhalf_pe_interpot = potential.evaluatePotentials(
            interpot, rhalf*apu.kpc, 0.).to_value(apu.km**2/apu.s**2)
        rhalf_pe_offset = rhalf_pe_star - rhalf_pe_interpot

        # Plotting
        this_fig_dir = os.path.join(fig_dir, df_type, df_version, str(z0_sid), 
            'merger_'+str(j+1))
        os.makedirs(this_fig_dir,exist_ok=True)

        fig,axs = plot_ELz(orbs, sample, energy-rhalf_pe_offset, fpot, 
            interpot)
        fig.tight_layout()
        figname = os.path.join(this_fig_dir,'energy_Lz.png')
        fig.savefig(figname, dpi=300)
        plt.close(fig)

        fig,axs = plot_beta_vdisp(orbs, sample)
        for ax in axs: ax.axvline(r_softening, linestyle='dashed', 
            linewidth=1., color='Grey')
        fig.tight_layout()
        figname = os.path.join(this_fig_dir,'velocity_dispersions.png')
        fig.savefig(figname, dpi=300)
        plt.close(fig)

        ### Osipkov-Merritt linear combination DF ###
        if verbose:
            msg = 'Sampling Osipkov-Merritt linear combination DF'
            logging.info(msg)
            print(msg)
        df_type = 'osipkov_merritt_2_combination'
        df_version = 'ra_N10_01_to_300_softening'
        try:
            # Load the distribution function and wrangle
            df_filename = os.path.join(df_fitting_dir,df_type,df_version,
                str(z0_sid),'merger_'+str(j+1),'df.pkl')
            with open(df_filename,'rb') as handle:
                data = pickle.load(handle)
            dfoms = data[0]
            ras = data[1]
            kom = data[2]
            ns = [int(kom*n_star), n_star-int(kom*n_star)]
            samples = []
            for k in range(len(dfoms)):
                dfoms[k] = pkin.reconstruct_anisotropic_df(dfoms[k], interpot, 
                    denspot)
                _sample = dfoms[k].sample(n=ns[k], rmin=rs.min()*apu.kpc*0.9)
                _sample = pkin.rotate_df_samples(_sample,stellar_halo_k,
                    stellar_halo_chi)
                samples.append(_sample)
            sample = putil.join_orbs(samples)
        except Exception as e:
            if verbose:
                msg = 'Caught an error when loading DF / sampling: '+str(e)+\
                    ' continuing'
                logging.info(msg)
                print(msg)
            continue
        
        _data_om2 = (z0_sid,
                    major_acc_sid,
                    major_mlpid,
                    j+1, # merger number
                    sample)
        sample_data_om2.append(_data_om2)

        # Plotting
        this_fig_dir = os.path.join(fig_dir, df_type, df_version, str(z0_sid), 
            'merger_'+str(j+1))
        os.makedirs(this_fig_dir,exist_ok=True)

        fig,axs = plot_ELz(orbs, sample, energy-rhalf_pe_offset, fpot,
            interpot)
        fig.tight_layout()
        figname = os.path.join(this_fig_dir,'energy_Lz.png')
        fig.savefig(figname, dpi=300)
        plt.close(fig)

        fig,axs = plot_beta_vdisp(orbs, sample)
        for ax in axs: 
            ax.axvline(r_softening, linestyle='dashed', 
                linewidth=1., color='Grey')
            for ra in ras: ax.axvline(ra, linestyle='dotted', 
                linewidth=1., color='Black')
        
        fig.tight_layout()
        figname = os.path.join(this_fig_dir,'velocity_dispersions.png')
        fig.savefig(figname, dpi=300)
        plt.close(fig)


# Save the data as a pickle
header = ['z0_sid','major_acc_sid','major_mlpid','merger_number','sample']
with open(os.path.join(analysis_dir,'sample_data_cb.pkl'),'wb') as handle:
    pickle.dump([header,sample_data_cb],handle)
with open(os.path.join(analysis_dir,'sample_data_om.pkl'),'wb') as handle:
    pickle.dump([header,sample_data_om],handle)
with open(os.path.join(analysis_dir,'sample_data_om2.pkl'),'wb') as handle:
    pickle.dump([header,sample_data_om2],handle)

# Also save as a structured array
dtype = [('z0_sid',int),
         ('major_acc_sid',int),
         ('major_mlpid',int),
         ('merger_number',int),
         ('sample',object)]
sample_data_cb_rec = np.array(sample_data_cb, dtype=dtype)
sample_data_om_rec = np.array(sample_data_om, dtype=dtype)
sample_data_om2_rec = np.array(sample_data_om2, dtype=dtype)
np.save(os.path.join(analysis_dir,'sample_data_cb.npy'),sample_data_cb_rec)
np.save(os.path.join(analysis_dir,'sample_data_om.npy'),sample_data_om_rec)
np.save(os.path.join(analysis_dir,'sample_data_om2.npy'),sample_data_om2_rec)